In [8]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Get the embeddings

In [ ]:
regions = os.listdir('/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/')
to_remove = ['logs', 'UKB-RAP', 'embeddings']
regions = [elem for elem in regions if elem not in to_remove]

In [10]:
embeddings_dir = f'/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/SFmarginal-SFinfant_left/*/hcp_random_epoch80_embeddings/full_embeddings.csv'
embeddings = pd.read_csv(embeddings_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/SFmarginal-SFinfant_left/*/hcp_random_epoch80_embeddings/full_embeddings.csv'

In [15]:
import glob
paths = glob.glob("/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/*/*/hcp_random_epoch80_embeddings/full_embeddings.csv")

In [21]:
ebd = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/FIP_labels.csv')

In [24]:
ebd

,Subject,Sex,Left_FIP,Right_FIP
0,100206,M,0,1
1,100307,F,1,1
2,100610,M,0,1
3,101309,M,1,1
4,101410,M,0,0
...,...,...,...,...
385,983773,M,1,1
386,984472,F,1,1
387,991267,M,1,1
388,993675,F,1,0


# Here, they were already saved in the same dir

In [23]:
dfs_dir = '/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/embeddings/ukb40_epoch80_embeddings'
dfs_dirs = os.listdir(dfs_dir)
dfs_dirs = [dfs_dir + '/' + elem for elem in dfs_dirs]
## ignore SFint-SR_right
dfs_dirs = [elem for elem in dfs_dirs if 'SFint-SR_right' not in elem]


In [24]:
embd_list = []

In [25]:
### TO REWRTIE, BECAUSE CAN CONCAT DIRECTLY THE NPYS
nb_subs = None

## load and rename the columns of each df
embd_list = []
n_dims = 256
for i, directory in enumerate(tqdm(dfs_dirs)):
    embd=pd.read_csv(directory)
    embd.columns = ['ID'] + [f'embd_{i}_{j}' for j in range(n_dims)]
    embd_list.append(embd.iloc[:nb_subs, :])
## merge all the dfs
embd = embd_list[0]
for i in tqdm(range(1, len(embd_list))):
    embd = pd.merge(embd, embd_list[i], on='ID', how='outer')
# drop all 'ID' columns but the first
embd = embd.loc[:, ~embd.columns.duplicated()]

100%|██████████| 54/54 [03:13<00:00,  3.58s/it]


In [26]:
model = LogisticRegression(solver='saga', penalty='elasticnet')
param_grid = {
    'C': [1, 10, 100, 1000, 10000, 100000],
    'l1_ratio': np.arange(0.5, 1.1, 0.1)
}
clf = GridSearchCV(model, param_grid, cv=5, refit=True, scoring='roc_auc', n_jobs=-1, verbose=2)

In [27]:
# get the labels
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/participants_sex_age.csv')
labels.columns = ['ID', 'Sex', 'Age']
embd_with_labels = embd.merge(labels, on='ID', how='inner')

In [28]:
Y = embd_with_labels['Sex'].to_numpy()
X = embd_with_labels.iloc[:, 1:-2].to_numpy()

In [ ]:
clf.fit(X, Y)
print(f'best params : {clf.best_params_}')
print(f'best score : {clf.best_score_}')

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [ ]:
## how much does the AUC depend on the grid search parameters ?
print(clf.cv_results_)

{'mean_fit_time': array([47.41689348, 47.59978347, 46.85675235, 47.21323705, 47.14969945,
       47.19448118,  0.56658678, 48.15658207, 47.86109061, 47.47113256,
       47.36005192, 46.93437409, 46.96833391,  0.31352787, 46.95876832,
       47.32715616, 47.51912127, 48.7087616 , 48.76176753, 48.81971765,
        0.2905592 , 48.6756372 , 47.19545302, 46.9460907 , 47.02589054,
       46.92865443, 46.8710032 ,  0.18247252, 48.69465504, 48.77238736,
       48.81611428, 48.83295441, 47.10978446, 40.04725957,  0.18971543,
       41.81674061, 41.48589706, 36.88923631, 35.12122946, 37.21280241,
       37.56897793,  0.19533796]), 'std_fit_time': array([5.49469426e-01, 7.00189874e-01, 1.18156782e-01, 5.74926370e-01,
       5.07605952e-01, 6.01839202e-01, 7.38295254e-02, 1.91016097e-01,
       5.46547229e-01, 8.51330365e-01, 4.10785167e-01, 1.01506799e-01,
       5.68997912e-02, 8.94249699e-02, 9.40343250e-02, 6.43741804e-01,
       8.05839476e-01, 8.20656876e-02, 1.17327272e-01, 8.00719386e-02,


# Try with PCA

In [ ]:
from sklearn.decomposition import PCA
# select n_features to keep 99.9% of the variance
pca = PCA(n_components=0.999)
X_pca = pca.fit_transform(X)
print(f'original shape: {X.shape}')
print(f'reduced shape: {X_pca.shape}')

original shape: (42423, 14080)
reduced shape: (42423, 1513)


In [ ]:
clf.fit(X_pca, Y)
print(f'best params : {clf.best_params_}')
print(f'best score : {clf.best_score_}')

## Open results

In [25]:
import pandas as pd
res = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/hcp_Left_OFC_global_classif_Nonesubs_100iter.csv')

In [26]:
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,29.930421,2.573159,0.013845,0.001630,0.001,0.0,"{'C': 0.001, 'l1_ratio': 0}",0.610857,0.626863,0.486875,0.537323,0.578614,0.568106,0.050865,19
1,44.834032,0.133778,0.013311,0.001413,0.001,0.2,"{'C': 0.001, 'l1_ratio': 0.2}",0.598569,0.686278,0.548938,0.575174,0.647296,0.611251,0.049545,3
2,42.028922,0.409847,0.012422,0.000444,0.001,0.4,"{'C': 0.001, 'l1_ratio': 0.4}",0.588479,0.693999,0.577558,0.589584,0.653227,0.620569,0.045384,1
3,39.853445,2.360050,0.012156,0.000200,0.001,0.6,"{'C': 0.001, 'l1_ratio': 0.6}",0.602096,0.694936,0.568436,0.582387,0.627058,0.614983,0.044572,2
4,38.736308,2.354798,0.011435,0.000979,0.001,0.8,"{'C': 0.001, 'l1_ratio': 0.8}",0.607052,0.647987,0.544849,0.575303,0.616261,0.598290,0.035366,5
5,38.685247,2.240549,0.010942,0.000734,0.001,1.0,"{'C': 0.001, 'l1_ratio': 1}",0.597807,0.588990,0.532783,0.578318,0.594542,0.578488,0.023792,10
6,29.657476,0.047708,0.011321,0.000181,0.010,0.0,"{'C': 0.01, 'l1_ratio': 0}",0.606278,0.627433,0.487943,0.538373,0.577902,0.567586,0.049763,23
7,66.014610,0.345056,0.011365,0.000266,0.010,0.2,"{'C': 0.01, 'l1_ratio': 0.2}",0.615014,0.643994,0.501660,0.548106,0.596780,0.581111,0.050499,9
8,55.891826,3.721758,0.011082,0.001030,0.010,0.4,"{'C': 0.01, 'l1_ratio': 0.4}",0.614513,0.654324,0.510209,0.551841,0.614714,0.589120,0.051319,8
9,54.418076,0.482473,0.012932,0.002713,0.010,0.6,"{'C': 0.01, 'l1_ratio': 0.6}",0.611471,0.660187,0.518876,0.557794,0.622938,0.594253,0.049955,7
